In [2]:
import numpy as np
from Instruments.Keithley_2612B import Keithley_2612B
from Instruments.Agilent_E3631A import Agilent_E3631A 
import time
import matplotlib.pyplot as plt

PS = Agilent_E3631A(5, 'p6')
PS.connect()

In [2]:
from Instruments.hp816x_instr import hp816x
import os
import sys  

hp = hp816x()
#hp.disconnect()
hp.connect('GPIB0::20::INSTR')
#hp.sweep()
hp.setAutorangeAll()

The mainframe is: HP8164A
Connected to the laser


In [2]:
from Instruments.hp816x_instr import hp816x
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

class sweepobj(object):
    
    def __init__(self,LMS, data_dir=os.getcwd()):
        self.LMS = LMS      
        self.data_dir = data_dir
        print(self.data_dir)   
        
    def setup_LMS(self):
        """Setup the Lightwave Measurement System for use."""
        self.LMS.setAutorangeAll()
    
    def wvl_sweep(self, wvl_start=1540, wvl_stop=1570, wvl_step=0.02, plot = True, filename=None):
        """Perform a wavelength sweep over the specified range."""
        
        # Init Instrument
        self.LMS.sweepUnit = 'dBm'
        self.LMS.sweepLaserOutput = 'highpower' # lowsse ou highpower
        self.LMS.sweepStartWvl = wvl_start * 1e-9
        self.LMS.sweepStopWvl = wvl_stop * 1e-9
        self.LMS.sweepStepWvl = wvl_step * 1e-9
        self.LMS.sweepInitialRange = 0
        self.LMS.sweepRangeDecrement = 20
        self.LMS.setPWMPowerUnit(2, 0, 'dBm')

        #Sweep
        wvl_sweep,pow_sweep = self.LMS.sweep()

        # Turn off the laser
        self.setup_LMS()
        
        # Plot the results
        f = plt.figure()
        if plot == True:
            plt.plot(wvl_sweep*1e9,pow_sweep.transpose()[0])
        if plot:
            plt.xlabel('Wavelength (nm)')
            plt.ylabel('Power (dBm)')
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            plt.show()
        
        # Save the results
        if filename == None:
            filename = raw_input("Enter the name of the file:")
        if not(filename==""):
            complete_name = self.data_dir + '\\' + filename 
            np.savetxt(complete_name + ".txt", (wvl_sweep,pow_sweep.transpose()[0]))
            f.savefig(complete_name + ".pdf")

In [ ]:
LMS = hp816x()
if LMS.connected == True:
    LMS.disconnect()
else:
    LMS.connect('GPIB0::20::INSTR')

so = sweepobj(LMS, 'C:\Users\Lab\Documents\MRM_QAM\Data\dec7')
so.wvl_sweep(1540,1552,0.1)

LMS.disconnect()

In [8]:
PS.source_voltage(1.4)


In [9]:
import numpy as np
from Instruments.Keithley_2612B import Keithley_2612B
from Instruments.Agilent_E3631A import Agilent_E3631A 
import time
import matplotlib.pyplot as plt

# DC sources objects
SMU_I = Keithley_2612B(0, 25, 'a')
SMU_Q = Keithley_2612B(0, 25, 'b')

# DC objects
#DC_I = Agilent_E3631A(1, '6')
#DC_Q = Agilent_E3631A(5, '6')

DC_I = Keithley_2612B(1, 26, 'a')
DC_Q = Keithley_2612B(1, 26, 'b')

PS = Agilent_E3631A(5, 'p6')

SMU_I.connect()
SMU_Q.connect()
DC_I.connect()
DC_Q.connect()
PS.connect()

In [14]:
# TUning to 1556.2 (Real)

# Set the rings reverse bias to 2V
SMU_I.source_voltage(0)
SMU_Q.source_voltage(0)
SMU_I.output_on()
SMU_Q.output_on()
# Set the heaters bias to 0
DC_I.source_voltage(0)#1.96v to align on the notch
DC_Q.source_voltage(4) ## Away from the carrier 4V
DC_I.output_on()
DC_Q.output_on()

# Set the phaseshift
PS.source_voltage(1.4)